# extract all csv from tensorboard

In [1]:
import os
import numpy as np
import pandas as pd

from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [15]:
def tabulate_events(dpath):

    final_out = {}
    for dname in os.listdir(dpath):
        print(f"Converting run {dname}",end="")
        try:
            ea = EventAccumulator(os.path.join(dpath, dname)).Reload()
            tags = ea.Tags()['scalars']
        except:
            continue
        out = {}

        for tag in tags:
            tag_values=[]
            steps=[]

            for event in ea.Scalars(tag):
                tag_values.append(event.value)
                steps.append(event.step)

            out[tag]=pd.DataFrame(data=dict(zip(steps,np.array([tag_values]).transpose())), columns=steps,index=['value'])

        if len(tags)>0:
            try:
                df= pd.concat(out.values(), keys=out.keys())
                df.to_csv(os.path.join(dump_path,'{dname}.csv'))
                print("- Done")
            except e:
                raise e
        else:
            print('- Not scalers to write')

        final_out[dname] = df

    return final_out

In [ ]:
path = "runs/norm_grad/shuffle_wo_normgrad" # where is the summary
steps = tabulate_events(path)

dump_path = 'mydump'
concate_filename = 'results.csv'

In [ ]:
pd.concat(steps.values(),keys=steps.keys()).to_csv(os.path.join(dump_path, concate_filename))